In [1]:
import requests
import json
import pandas as pd
from io import StringIO 
import time
import matplotlib.pyplot as plt
#token expires
token = 'eyJ4NXQiOiJZV0kxTTJZNE1qWTNOemsyTkRZeU5XTTRPV014TXpjek1UVmhNbU14T1RSa09ETXlOVEE0Tnc9PSIsImtpZCI6ImdhdGV3YXlfY2VydGlmaWNhdGVfYWxpYXMiLCJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiJBbGVleFY2QGNhcmJvbi5zdXBlciIsImFwcGxpY2F0aW9uIjp7Im93bmVyIjoiQWxlZXhWNiIsInRpZXJRdW90YVR5cGUiOm51bGwsInRpZXIiOiJVbmxpbWl0ZWQiLCJuYW1lIjoiRGVmYXVsdEFwcGxpY2F0aW9uIiwiaWQiOjEzNDgyLCJ1dWlkIjoiYjIwZWI3NWQtZDcxMS00MjJlLThlMzAtMWMzMzgwY2FlNzVjIn0sImlzcyI6Imh0dHBzOlwvXC9wb3J0YWlsLWFwaS5tZXRlb2ZyYW5jZS5mcjo0NDNcL29hdXRoMlwvdG9rZW4iLCJ0aWVySW5mbyI6eyI1MFBlck1pbiI6eyJ0aWVyUXVvdGFUeXBlIjoicmVxdWVzdENvdW50IiwiZ3JhcGhRTE1heENvbXBsZXhpdHkiOjAsImdyYXBoUUxNYXhEZXB0aCI6MCwic3RvcE9uUXVvdGFSZWFjaCI6dHJ1ZSwic3Bpa2VBcnJlc3RMaW1pdCI6MCwic3Bpa2VBcnJlc3RVbml0Ijoic2VjIn19LCJrZXl0eXBlIjoiUFJPRFVDVElPTiIsInN1YnNjcmliZWRBUElzIjpbeyJzdWJzY3JpYmVyVGVuYW50RG9tYWluIjoiY2FyYm9uLnN1cGVyIiwibmFtZSI6IkRvbm5lZXNQdWJsaXF1ZXNDbGltYXRvbG9naWUiLCJjb250ZXh0IjoiXC9wdWJsaWNcL0RQQ2xpbVwvdjEiLCJwdWJsaXNoZXIiOiJhZG1pbl9tZiIsInZlcnNpb24iOiJ2MSIsInN1YnNjcmlwdGlvblRpZXIiOiI1MFBlck1pbiJ9XSwiZXhwIjoxODEyOTgyODUxLCJ0b2tlbl90eXBlIjoiYXBpS2V5IiwiaWF0IjoxNzE4MzEwMDUxLCJqdGkiOiIxYTBhOGRkNy0xMGVhLTRjZWItOTE4Ni03NjcxMzczYjY3Y2QifQ==.i4neINRr77mI5vc4zriCSlX5LW9f-aT6Rty0JGYqLnuU2j4b89LIYcmtlg53S3fV03YS3t4pcrSMR2pyC6OU4v98ejkhXZ2W9h40ivN50Ye-NOCRksFrtLaMDU7mZnfNG6dCLcx9h0lw6xaVjVV1UD1oR9I61QoV37M4kJXanGghBAPq2bXd98_Y1aNf2O49BwhegKZ3w7iAQJbeLCso36--OskBqbx6XNpob8M1KAAaCBqwCPT6me6rg7AYpyzILLmP4Kn8zukcvV4S0IY5mx7PlEVYS5fFl3nn6zbdzoYsVdU7vPvdGbdBuDLJc6WElwE4nI3Okfrs1OUk2oM2iw=='
headers = {'apikey': token}

In [2]:
start = ['2023-01-01T00:00:00Z', '2024-01-01T00:00:00Z']
end = ['2023-12-31T00:00:00Z', '2024-06-12T00:00:00Z']
nouvelle_aquitaine = [16,17,19,23,24,33, 40,47,64,79,86,87]

In [3]:
def get_first_two_digits(n):
    str_n = str(abs(n))  # Use abs to handle negative numbers correctly
    first_two_digits = str_n[:2]
    return int(first_two_digits)
def get_station_list(departement):
    url = f'https://public-api.meteofrance.fr/public/DPClim/v1/liste-stations/quotidienne?id-departement={departement}'
    r = requests.get(url, headers=headers)
    df = pd.DataFrame(r.json())
    return df
def retreive_data(id_request):
    url = f'https://public-api.meteofrance.fr/public/DPClim/v1/commande/fichier?id-cmde={id_request}'
    while True:
        r = requests.get(url, headers=headers)
        if r.status_code == 201:
            content = r.content.decode('utf-8')
            data = StringIO(content)
            df = pd.read_csv(data, delimiter=';')
            df['DEPARTEMENT'] = df['POSTE'].apply(get_first_two_digits)
            return df
        elif r.status_code == 204:
            time.sleep(5)
        elif r.status_code == 502:
            #print('502 found, sleeping...')
            time.sleep(5)
        else:
            print(f"Unexpected status code: {r.status_code}")
            break
def request_data(id_station_list, start_date, end_date):
    df_list = []
    for station in id_station_list:
        print(station)
        for start, end in zip(start_date, end_date):
            url = f'https://public-api.meteofrance.fr/public/DPClim/v1/commande-station/quotidienne?id-station={station}&date-deb-periode={start}&date-fin-periode={end}'
            while True:
                r = requests.get(url, headers=headers)  
                if r.status_code == 202:
                    returnedData = retreive_data(r.json()['elaboreProduitAvecDemandeResponse']['return'])
                    #print(f"Station : {station} | {start} - {end} | {r} OK")
                    #print('--------------')
                    df_list.append(returnedData)    
                    break
                else:
                    #print(f"Can't find data for {station}, {start} to {end}, response : {r.status_code}, sleeping...")
                    time.sleep(5)
    df = pd.concat(df_list).reset_index(drop=True)                    
    return df

In [5]:
list_nouvelle_aquitaine = []
for dep in nouvelle_aquitaine:
    data = get_station_list(dep)
    df = request_data(data[data['posteOuvert']]['id'], start, end)
    list_nouvelle_aquitaine.append(df)
df_nouvelle_aquitaine = pd.concat(list_nouvelle_aquitaine).reset_index(drop=True)
df_nouvelle_aquitaine

16028001
16078001
16089001
16113001
16193001
16225001
16256001
16279001
16363001
16366001
16390001
16403001
17093002
17111001
17218001
17224001
17243002
17248001
17268002
17300009
17306004
17308001
17318001
17323001
17339002
17353001
17415003
17477001
19010001
19016001
19031008
19033001
19034001
19073006
19121002
19125001
19136002
19139001
19147001
19164001
19201001
19237001
19264001
19272001
19275001
19276006
19288004
23008004
23013001
23021001
23025001
23030004
23031002
23046001
23067001
23069002
23075001
23079002
23089001
23090003
23113001
23155001
23176001
23206002
23209001
24035003
24037005
24064001
24138004
24152001
24172001
24226001
24280001
24291001
24452001
24453001
24516002
24547002
24550003
33042001
33042005
33063001
33080001
33095001
33116001
33122003
33195001
33214002
33236002
33281001
33314005
33394002
33415001
33482001
33484001
33494001
33504001
33529001
33540001
33550001
40031001
40033001
40046001
40065002
40087001
40088001
40134001
40164004
40192001
40218002
40227001
4

,POSTE,DATE,RR,DRR,TN,HTN,TX,HTX,TM,TMNX,...,ECLAIR,ETPMON,ETPGRILLE,UV,TMERMAX,TMERMIN,HNEIGEF,NEIGETOTX,NEIGETOT06,DEPARTEMENT
0,16028001,20230101,"1,2",NaN,"10,6",430.0,"17,8",1343.0,"13,5","14,20",...,NaN,NaN,"2,2",NaN,NaN,NaN,NaN,NaN,NaN,16
1,16028001,20230102,"3,2",NaN,"8,6",1751.0,"14,7",1334.0,"10,2","11,65",...,NaN,NaN,"0,9",NaN,NaN,NaN,NaN,NaN,NaN,16
2,16028001,20230103,"0,2",NaN,"1,9",709.0,"13,8",1321.0,"6,6","7,85",...,NaN,NaN,"0,4",NaN,NaN,NaN,NaN,NaN,NaN,16
3,16028001,20230104,"1,0",NaN,"5,6",2310.0,"12,6",1358.0,"9,5","9,10",...,NaN,NaN,"0,3",NaN,NaN,NaN,NaN,NaN,NaN,16
4,16028001,20230105,"0,2",NaN,"10,9",517.0,"14,7",1233.0,"11,5","12,80",...,NaN,NaN,"0,8",NaN,NaN,NaN,NaN,NaN,NaN,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113730,87187003,20240608,"0,0",NaN,"14,1",22.0,"25,8",1249.0,"19,6","19,95",...,NaN,"3,5","3,5",NaN,NaN,NaN,NaN,NaN,NaN,87
113731,87187003,20240609,"7,4",NaN,"14,0",1421.0,"17,3",1825.0,"14,8","15,65",...,NaN,"1,6","1,6",NaN,NaN,NaN,NaN,NaN,NaN,87
113732,87187003,20240610,"0,0",NaN,"9,2",333.0,"20,9",1402.0,"15,5","15,05",...,NaN,"4,1","4,1",NaN,NaN,NaN,NaN,NaN,NaN,87
113733,87187003,20240611,"0,2",NaN,"8,3",143.0,"19,1",1534.0,"14,0","13,70",...,NaN,"3,6","3,6",NaN,NaN,NaN,NaN,NaN,NaN,87


In [6]:
def get_market_year(row):
    year = row['Year']
    week = row['Week']
    culture = row['Culture']
    if culture in ['Blé tendre', 'Blé dur']:
        if week >= 36:
            market_year = f"{year}/{year + 1}"
        else:
            market_year = f"{year - 1}/{year}"
    else:
        if 9 <= week <= 49:
            market_year = f"{year - 1}/{year}"
        else:
            market_year = f"{year - 1}/{year}"
    return market_year

def get_market_year_ble(row):
    year = row['Year']
    week = row['Week']
    if week >= 36:
        market_year = f"{year}/{year + 1}"
    else:
        market_year = f"{year - 1}/{year}"
    return market_year

In [12]:
dfMeteo = df_nouvelle_aquitaine
dfMeteo['DATE'] = pd.to_datetime(dfMeteo['DATE'], format='%Y%m%d')
dfMeteo['TX'] = dfMeteo['TX'].replace(',', '.', regex=True).astype(float)
dfMeteo['TN'] = dfMeteo['TN'].replace(',', '.', regex=True).astype(float)
dfMeteo['RR'] = dfMeteo['RR'].replace(',', '.', regex=True).astype(float)
dfMeteo['DG'] = dfMeteo['DG'].replace(',', '.', regex=True).astype(float)
dfMeteo['Year'] = dfMeteo['DATE'].dt.year
dfMeteo['Week'] = dfMeteo['DATE'].dt.isocalendar().week
dfMeteo['MarketYear'] = dfMeteo.apply(get_market_year_ble, axis=1)
dfMeteo = dfMeteo[['POSTE', 'DEPARTEMENT', 'DATE', 'MarketYear', 'TX', 'TN', 'RR', 'DG']]

dfMeteoMoyenneNA = dfMeteo.groupby('DATE').agg({
    'MarketYear': 'first',
    'TX': 'mean',
    'TN': 'mean',
    'RR': 'mean',
    'DG': 'mean',
})
dfMeteoMoyenneNA = dfMeteoMoyenneNA[dfMeteoMoyenneNA['MarketYear'] == '2023/2024'].iloc[1:]

In [13]:
dfMeteoMoyenneNA

,MarketYear,TX,TN,RR,DG
DATE,,,,,
2023-09-04,2023/2024,34.631132,18.033019,0.015166,0.0
2023-09-05,2023/2024,32.342925,17.883019,0.019811,0.0
2023-09-06,2023/2024,32.433019,17.747170,0.024528,0.0
2023-09-07,2023/2024,32.931132,16.920755,0.021801,0.0
2023-09-08,2023/2024,33.178302,16.793396,0.029717,0.0
...,...,...,...,...,...
2024-06-08,2023/2024,25.059906,15.732547,10.668396,0.0
2024-06-09,2023/2024,21.006132,14.141038,2.298578,0.0
2024-06-10,2023/2024,21.225000,10.271226,1.078774,0.0


In [14]:
MeteoRRMarketYearCumulative = dfMeteoMoyenneNA.groupby('MarketYear')['RR'].cumsum()
MeteoRRMarketYearCumulative

DATE
2023-09-04       0.015166
2023-09-05       0.034977
2023-09-06       0.059505
2023-09-07       0.081306
2023-09-08       0.111023
                 ...     
2024-06-08    1172.746214
2024-06-09    1175.044792
2024-06-10    1176.123566
2024-06-11    1177.491018
2024-06-12    1177.873957
Name: RR, Length: 283, dtype: float64